In [51]:
from mbi import Dataset, FactoredInference, Factor, GraphicalModel, Domain, CliqueVector, RegionGraph, FactorGraph
import numpy as np
import itertools
from scipy import sparse
import pandas as pd


In [52]:
def get_random_cliques(attrs, size, number, prng=np.random):
    allcl = list(itertools.combinations(attrs, size))
    idx = prng.choice(len(allcl), number, replace=False)
    return [allcl[i] for i in idx]


In [64]:
attrs = ['A', 'B', 'C', 'D', 'E']
domain = Domain(attrs, [2]*len(attrs))
cliques = [('D', 'E'), ('C', 'D'), ('A', 'C'), ('B', 'E'), ('A', 'D')]
model = RegionGraph(domain, cliques, iters=1000, convex=True, minimal=True)
potentials = CliqueVector.random(domain, model.cliques)*3
marginals0 = model.belief_propagation(potentials)
#marginals1 = model.optimize_bethe(potentials)

In [145]:
#cliques2 = cliques# + [('A','B')]
cliques2 = list(itertools.combinations(attrs, 2))
model2 = RegionGraph(domain, cliques2, iters=1000, convex=True, minimal=True)
for key in model2.counting_numbers:
    if not key in model.counting_numbers:
        model2.counting_numbers[key] = 0.000001
potentials2 = CliqueVector.zeros(domain, model2.cliques)
potentials2.combine(potentials)
marginals2 = model2.belief_propagation(potentials2)
#marginals3 = model2.optimize_bethe(potentials2)

In [146]:
for cl in cliques:
    print(marginals0[cl].datavector())
    #print(marginals1[cl].datavector())
    print(marginals2[cl].datavector())
    #print(marginals3[cl].datavector())
    print()
    
print(marginals2[('A','B')].datavector())

[0.1625758  0.26524909 0.46468453 0.10749058]
[0.16257578 0.26524915 0.46468447 0.1074906 ]

[0.2087245  0.18049793 0.2191004  0.39167717]
[0.20872454 0.18049795 0.21910039 0.39167712]

[0.33555426 0.31799587 0.05366817 0.2927817 ]
[0.33555429 0.31799576 0.0536682  0.29278175]

[0.42738935 0.2606159  0.19987098 0.11212377]
[0.42738887 0.26061571 0.19987139 0.11212404]

[0.25569157 0.39785856 0.17213332 0.17431654]
[0.25569155 0.39785849 0.17213338 0.17431658]

[0.44964542 0.20390462 0.23835915 0.1080908 ]


In [147]:
phi = potentials2[('A','B')]
msg1 = model2.messages[(('A',),('A','B'))]
msg2 = model2.messages[(('B',),('A','B'))]
c = model2.counting_numbers[('A','B')]
belief = ((phi + msg1 + msg2)/c)
mu = (belief - belief.logsumexp()).exp()
print(mu.datavector())
print(belief.datavector())
print(msg1.datavector(), msg2.datavector(), phi.datavector())

[0.44964542 0.20390462 0.23835915 0.1080908 ]
[-1386293.64837616 -1386294.43918313 -1386294.28305691 -1386295.07386388]
[-0.69314686 -0.6931475 ] [-0.69314679 -0.69314758] [0. 0. 0. 0.]


In [134]:
pot = potentials2
messages = model2.messages
self = model2
belief = (pot[r] + sum(messages[c,r] for c in self.children[r]) - sum(messages[r,p] for p in self.parents[r])) / c
mu = (belief - belief.logsumexp()).exp()
print(mu.datavector())

[0.44959582 0.20394581 0.23834172 0.10811665]


In [142]:
model2.messages[(('A',),('A','B'))].datavector()

array([-0.69311545, -0.69317891])

In [144]:
msg2.datavector()

array([-0.69310766, -0.69318671])